# Stereo-seq CellBin Analysis (Cleaned Notebook)

This notebook is a cleaned, publication- and GitHub-ready version of the original analysis.
It removes Chinese inline comments, replaces hard-coded personal paths with a configurable input path,
and adds English explanations for reproducibility.

**How to use**
1. Put your input `.h5ad` in `data/raw/`
2. Set `INPUT_H5AD` below
3. Run cells top-to-bottom

> Note: CellBin units are not true single cells. Interpret "cell types" as **cell-type–associated CellBin units** inferred from markers.


In [ ]:
from matplotlib import rcParams
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from scipy import sparse
from scipy.spatial import KDTree
from scipy.stats import chi2_contingency, fisher_exact
from scipy.stats import fisher_exact, chi2_contingency
from scipy.stats import mannwhitneyu
from scipy.stats import ranksums
import matplotlib
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import os
import pandas as pd
import scanpy as sc
import seaborn as sns
import statsmodels.formula.api as smf

# -----------------------------------------------------------------------------
# Configuration
# -----------------------------------------------------------------------------
# Path to your input AnnData object exported from Stereo-seq/CellBin.
# Do NOT commit raw data to GitHub. Keep it under data/raw/ and ignore it via .gitignore.
INPUT_H5AD = "/home/yuanyanyao/Downloads/数据/stereo-seq/raw_data/Y01504E1.cellbin_1.0.adjusted.h5ad"

# Output directory for figures/tables (safe to commit if small; otherwise ignore results/)
OUTDIR = "results"
os.makedirs(OUTDIR, exist_ok=True)

# Global Scanpy plotting parameters
sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=120, facecolor="white")


In [ ]:
# -----------------------------------------------------------------------------
# Cell 00: Load input AnnData (.h5ad)
# -----------------------------------------------------------------------------
# Purpose:
# - Load the Stereo-seq/CellBin AnnData object.
# - Sanity-check dimensions and metadata fields.

sc.settings.verbosity = 3
sc.settings.set_figure_params(dpi=80, facecolor='white')

adata = sc.read_h5ad(INPUT_H5AD)
print(adata)


In [ ]:
# -----------------------------------------------------------------------------
# Cell 01: Inspect UMAP / cluster structure
# -----------------------------------------------------------------------------
# Purpose:
# - Visual QC: verify cluster separation and potential artifacts.
# - Use consistent color keys (e.g., `leiden`, `cell_type`).

sc.pl.umap(adata, color='leiden')


In [ ]:
# -----------------------------------------------------------------------------
# Cell 02: Inspect UMAP / cluster structure
# -----------------------------------------------------------------------------
# Purpose:
# - Visual QC: verify cluster separation and potential artifacts.
# - Use consistent color keys (e.g., `leiden`, `cell_type`).

sc.pl.umap(adata, color=['leiden', 'total_counts', 'pct_counts_mt'])
sc.pl.spatial(adata, color='leiden', spot_size=10)


In [ ]:
# -----------------------------------------------------------------------------
# Cell 03: Run Leiden clustering
# -----------------------------------------------------------------------------
# Purpose:
# - Compute a graph-based clustering on the current embedding/graph.
# - Tune `resolution` to control cluster granularity.

sc.tl.leiden(adata, resolution=1.5, key_added='leiden')


In [ ]:
# -----------------------------------------------------------------------------
# Cell 04: Spatial visualization / sanity checks
# -----------------------------------------------------------------------------
# Purpose:
# - Plot spatial distributions to confirm orientation, tissue mask, and region logic.
# - Always cross-check against histology if available.

clusters = adata.obs['leiden'].cat.categories

for cluster_id in clusters:
    print(f"Plotting Cluster {cluster_id}...")

    sc.pl.spatial(
        adata,
        color='leiden',
        groups=[cluster_id],
        spot_size=30,

        title=f"Spatial Dist: Cluster {cluster_id}"
    )


    plt.show()


In [ ]:
# -----------------------------------------------------------------------------
# Cell 05: Rank marker genes (scanpy.tl.rank_genes_groups)
# -----------------------------------------------------------------------------
# Purpose:
# - Identify cluster-enriched genes to support cell-type labeling.
# - Review top markers and confirm they match expected biology.

adata.var['gene_id'] = adata.var_names

adata.var['real_gene_name'] = adata.var['real_gene_name'].fillna(adata.var['gene_id']).astype(str)


new_gene_names = adata.var['real_gene_name'].values

unique_names, inverse_indices = np.unique(new_gene_names, return_inverse=True)
n_new_genes = len(unique_names)
n_old_genes = len(new_gene_names)

row_ind = np.arange(n_old_genes)
col_ind = inverse_indices
data = np.ones(n_old_genes)

agg_matrix = sparse.csr_matrix((data, (row_ind, col_ind)), shape=(n_old_genes, n_new_genes))


X_new = adata.X @ agg_matrix


new_var = pd.DataFrame(index=unique_names)
new_var['gene_id'] = new_var.index

adata_new = sc.AnnData(X=X_new, obs=adata.obs, var=new_var)
adata_new.obsm = adata.obsm
adata_new.uns = adata.uns

del agg_matrix

# ==========================================
# ==========================================

sc.pp.log1p(adata_new)

adata_new.raw = adata_new


sc.tl.rank_genes_groups(adata_new, groupby='leiden', method='wilcoxon')


In [ ]:
# -----------------------------------------------------------------------------
# Cell 06: Rank marker genes (scanpy.tl.rank_genes_groups)
# -----------------------------------------------------------------------------
# Purpose:
# - Identify cluster-enriched genes to support cell-type labeling.
# - Review top markers and confirm they match expected biology.

print(f"Max after log1p: {adata_new.X.max():.2f}")

print("First 10 gene names:", adata_new.var_names[:10].tolist())

sc.pl.rank_genes_groups_dotplot(
    adata_new,
    n_genes=3,
    standard_scale='var',
    cmap='bwr',
    title="Marker Genes (Aggregated & Logged)"
)


In [ ]:
# -----------------------------------------------------------------------------
# Cell 07: Analysis step
# -----------------------------------------------------------------------------
# Purpose:
# - Intermediate analysis step from the original notebook.
# - Consider refactoring into functions if reused.

plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False


marker_genes = {
    'Pan-Neuron': ['Rbfox3', 'Snap25', 'Syt1'],

    'Excitatory': ['Slc17a7', 'Slc17a6', 'Camk2a'],

    'Inhibitory': ['Gad1', 'Gad2', 'Slc32a1'],

    'DA_Neuron': ['Th', 'Slc6a3', 'Ddc', 'Slc18a2'],
    'DA_TF': ['Nr4a2', 'Pitx3', 'En1', 'Lmx1b', 'Foxa2'],

    'Cholinergic': ['Chat', 'Slc18a3', 'Slc5a7'],

    'Serotonergic': ['Tph2', 'Slc6a4', 'Fev'],

    'Noradrenergic': ['Dbh', 'Slc6a2', 'Phox2b'],

    'Astrocyte': ['Gfap', 'Aqp4', 'Aldh1l1', 'S100b', 'Slc1a3'],

    'OPC': ['Pdgfra', 'Cspg4', 'Olig1'],
    'Oligo': ['Mbp', 'Mog', 'Plp1', 'Mag'],

    'Microglia': ['Cx3cr1', 'P2ry12', 'Tmem119', 'Aif1', 'Csf1r'],

    'Endothelial': ['Pecam1', 'Cldn5', 'Flt1', 'Vwf'],
    'Pericyte': ['Pdgfrb', 'Rgs5', 'Kcnj8'],
    'VSMC': ['Acta2', 'Myh11', 'Tagln'],

    'Ependymal': ['Foxj1', 'Pifo', 'Dynlrb2'],
    'Immune': ['Ptprc', 'Cd74', 'H2-Aa'],
}

marker_list = []
for cell_type, genes in marker_genes.items():
    marker_list.extend(genes)

marker_list = list(dict.fromkeys(marker_list))


sc.pl.dotplot(
    adata_new,
    var_names=marker_genes,
    groupby='leiden',
    dendrogram=True,
    standard_scale='var',
    cmap='Reds',
    figsize=(18, 8),
    save='_brain_markers.pdf'
)


In [ ]:
# -----------------------------------------------------------------------------
# Cell 08: Rank marker genes (scanpy.tl.rank_genes_groups)
# -----------------------------------------------------------------------------
# Purpose:
# - Identify cluster-enriched genes to support cell-type labeling.
# - Review top markers and confirm they match expected biology.

markers_df = pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(20)

print(markers_df)


markers_df.to_csv("all_clusters_markers.csv")


In [ ]:
# -----------------------------------------------------------------------------
# Cell 09: Rank marker genes (scanpy.tl.rank_genes_groups)
# -----------------------------------------------------------------------------
# Purpose:
# - Identify cluster-enriched genes to support cell-type labeling.
# - Review top markers and confirm they match expected biology.

result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names

for group in groups:
    genes = [result['names'][i][group] for i in range(10)]
    print(f"Cluster {group}: {', '.join(genes)}")


In [ ]:
# -----------------------------------------------------------------------------
# Cell 10: Inspect UMAP / cluster structure
# -----------------------------------------------------------------------------
# Purpose:
# - Visual QC: verify cluster separation and potential artifacts.
# - Use consistent color keys (e.g., `leiden`, `cell_type`).

cluster_annotation = {str(i): "" for i in range(36)}


cluster_annotation = {
    '1': 'Excitatory_Neuron',
    '4': 'Excitatory_Neuron',
    '6': 'Excitatory_Neuron',
    '18': 'Hippocampal_Neuron',
    '22': 'Hippocampal_Neuron',
    '23': 'Inhibitory_Neuron',
    '24': 'Hippocampal_Neuron',
    '29': 'DA_Neuron',
    '34': 'Serotonergic_Neuron',

    '2': 'Astrocyte',
    '3': 'Oligodendrocyte',
    '12': 'Astrocyte',
    '15': 'Oligodendrocyte',
    '16': 'Oligodendrocyte',
    '17': 'Oligodendrocyte',
    '19': 'Microglia',
    '25': 'Choroid_Plexus',
    '26': 'Unknown',
    '27': 'Choroid_Plexus',
    '28': 'Astrocyte',
    '30': 'Astrocyte',
    '35': 'Ependymal/Progenitor',


    '0': '',
    '7': '',
    '8': '',
    '31': '',
    '32': '',

    '5': '',
    '9': '',
    '10': '',
    '11': '',
    '13': '',
    '14': '',
    '20': '',
    '21': '',

    '33': '',
}

adata_new.obs['cell_type'] = adata_new.obs['leiden'].map(cluster_annotation)

print("Clusters to resolve：")
print(adata_new.obs[adata_new.obs['cell_type'] == '']['leiden'].unique())
adata_new.obs['cell_type'] = adata_new.obs['leiden'].map(cluster_annotation)


adata_new.obs['cell_type'] = adata_new.obs['leiden'].map(cluster_annotation)

print(adata_new.obs['cell_type'].value_counts())

sc.pl.umap(adata_new, color='cell_type', legend_loc='on data', title='Brain Cell Types')


In [ ]:
# -----------------------------------------------------------------------------
# Cell 11: Subcluster unresolved / mixed clusters
# -----------------------------------------------------------------------------
# Purpose:
# - Focus on ambiguous clusters and re-cluster for finer separation.
# - Recompute HVGs within the subset to avoid bias from global HVGs.

adata_unresolved = adata_new[adata_new.obs['cell_type'] == ''].copy()

print(f"Remaining {adata_unresolved.n_obs} units require subclustering-based annotation。")


In [ ]:
# -----------------------------------------------------------------------------
# Cell 12: Run Leiden clustering
# -----------------------------------------------------------------------------
# Purpose:
# - Compute a graph-based clustering on the current embedding/graph.
# - Tune `resolution` to control cluster granularity.

sc.pp.highly_variable_genes(adata_unresolved, min_mean=0.0125, max_mean=3, min_disp=0.5)

sc.tl.pca(adata_unresolved, svd_solver='arpack')

sc.pp.neighbors(adata_unresolved, n_neighbors=15, n_pcs=30)

sc.tl.umap(adata_unresolved)

sc.tl.leiden(adata_unresolved, resolution=0.8, key_added='sub_leiden')

print("Re-clustering finished！")


In [ ]:
# -----------------------------------------------------------------------------
# Cell 13: Subcluster unresolved / mixed clusters
# -----------------------------------------------------------------------------
# Purpose:
# - Focus on ambiguous clusters and re-cluster for finer separation.
# - Recompute HVGs within the subset to avoid bias from global HVGs.

sc.pl.spatial(
        adata_unresolved,
        color='sub_leiden',
        spot_size=30,

        title=f"Spatial Dist: Cluster {cluster_id}"
    )


In [ ]:
# -----------------------------------------------------------------------------
# Cell 14: Rank marker genes (scanpy.tl.rank_genes_groups)
# -----------------------------------------------------------------------------
# Purpose:
# - Identify cluster-enriched genes to support cell-type labeling.
# - Review top markers and confirm they match expected biology.

sc.tl.rank_genes_groups(adata_unresolved, 'sub_leiden', method='wilcoxon')

result = adata_unresolved.uns['rank_genes_groups']
groups = result['names'].dtype.names

print("=== Marker genes for new subclusters ===")
for group in groups:
    genes = [result['names'][i][group] for i in range(5)]
    print(f"Sub-cluster {group}: {', '.join(genes)}")

sc.pl.rank_genes_groups_dotplot(adata_unresolved, n_genes=3, standard_scale='var')


In [ ]:
# -----------------------------------------------------------------------------
# Cell 15: Subcluster unresolved / mixed clusters
# -----------------------------------------------------------------------------
# Purpose:
# - Focus on ambiguous clusters and re-cluster for finer separation.
# - Recompute HVGs within the subset to avoid bias from global HVGs.

sub_clusters = adata_unresolved.obs['sub_leiden'].cat.categories

for cluster_id in sub_clusters:
    print(f"Plotting Cluster {cluster_id}...")

    sc.pl.spatial(
        adata_unresolved,
        color='sub_leiden',
        groups=[cluster_id],
        spot_size=30,

        title=f"Spatial Dist: Cluster {cluster_id}"
    )


    plt.show()


In [ ]:
# -----------------------------------------------------------------------------
# Cell 16: Inspect UMAP / cluster structure
# -----------------------------------------------------------------------------
# Purpose:
# - Visual QC: verify cluster separation and potential artifacts.
# - Use consistent color keys (e.g., `leiden`, `cell_type`).

adata_new.obs['cell_type'] = adata_new.obs['cell_type'].astype(str)

sub_cluster_names = {
    '0': 'Neuron_Nap1l5',
    '1': 'Excitatory_Ctx_Deep',
    '2': 'Vascular/Noise',
    '3': 'White_Matter',
    '4': 'White_Matter',
    '5': 'Excitatory_Piriform',
    '6': 'Dentate_Gyrus',
    '7': 'Thalamus_Pcp4',
}

new_labels = adata_unresolved.obs['sub_leiden'].map(sub_cluster_names).to_dict()
new_labels_series = pd.Series(new_labels)


adata_new.obs['cell_type'].update(new_labels_series)

print("✅ Annotation updated successfully！")

# ==========================================
# ==========================================

garbage_list = ['Vascular/Noise', 'White_Matter', 'Low_Quality/Mixed']

adata_final = adata_new[~adata_new.obs['cell_type'].isin(garbage_list)].copy()

adata_final.obs['cell_type'] = adata_final.obs['cell_type'].astype('category')

if hasattr(adata_final.obs['cell_type'], 'cat'):
    adata_final.obs['cell_type'] = adata_final.obs['cell_type'].cat.remove_unused_categories()

# ==========================================
# ==========================================
print(f"\nOriginal n_obs: {adata_new.n_obs}")
print(f"Filtered n_obs: {adata_final.n_obs}")
print(f"Removed {adata_new.n_obs - adata_final.n_obs} background/noise units。")

print("\nFinal retained cell types：")
print(adata_final.obs['cell_type'].value_counts())

sc.pl.umap(adata_final, color='cell_type', title='Final Annotated & Cleaned', legend_loc='on data', frameon=False)


In [ ]:
# -----------------------------------------------------------------------------
# Cell 17: Analysis step
# -----------------------------------------------------------------------------
# Purpose:
# - Intermediate analysis step from the original notebook.
# - Consider refactoring into functions if reused.

adata_final.obs['cell_type'] = adata_final.obs['cell_type'].astype(str)

adata_final.obs.loc[adata_final.obs['cell_type'] == 'Immune_Peripheral', 'cell_type'] = 'Unknown'

adata_final.obs['cell_type'] = adata_final.obs['cell_type'].astype('category')


In [ ]:
# -----------------------------------------------------------------------------
# Cell 19: Inspect UMAP / cluster structure
# -----------------------------------------------------------------------------
# Purpose:
# - Visual QC: verify cluster separation and potential artifacts.
# - Use consistent color keys (e.g., `leiden`, `cell_type`).

matplotlib.rcParams['font.family']      = 'serif'
matplotlib.rcParams['font.serif']       = ['Times New Roman'] + matplotlib.rcParams['font.serif']
matplotlib.rcParams['savefig.dpi']      = 600
matplotlib.rcParams['figure.dpi']       = 600
matplotlib.rcParams['axes.labelsize']   = 10
matplotlib.rcParams['xtick.labelsize']  = 9
matplotlib.rcParams['ytick.labelsize']  = 9
matplotlib.rcParams['legend.fontsize']  = 9
matplotlib.rcParams['legend.frameon']   = False

sc.pl.umap(adata_final, color='cell_type',
           frameon=False,
           title='Cell-type distribution',
           save='umap_celltype.pdf')
sc.pl.umap(adata_final, color='cell_type',
           title='Cell-type distribution',
           frameon=False,
           save='umap_celltype.png')   # 600 dpi PNG


In [ ]:
# -----------------------------------------------------------------------------
# Cell 20: Spatial visualization / sanity checks
# -----------------------------------------------------------------------------
# Purpose:
# - Plot spatial distributions to confirm orientation, tissue mask, and region logic.
# - Always cross-check against histology if available.

if 'spatial_original' not in adata_final.obsm.keys():
    adata_final.obsm['spatial_original'] = adata_final.obsm['spatial'].copy()

coords = adata_final.obsm['spatial_original'].copy()


coords[:, [0, 1]] = coords[:, [1, 0]]
coords[:, 0] = -coords[:, 0]


adata_final.obsm['spatial'] = coords

sc.pl.spatial(adata_final, color='cell_type', spot_size=20, title='Check Orientation')


In [ ]:
# -----------------------------------------------------------------------------
# Cell 21: Spatial visualization / sanity checks
# -----------------------------------------------------------------------------
# Purpose:
# - Plot spatial distributions to confirm orientation, tissue mask, and region logic.
# - Always cross-check against histology if available.

sc.pl.spatial(adata_final, color='cell_type', spot_size=30, show=False)

ax = plt.gca()

handles, labels = ax.get_legend_handles_labels()

ax.legend(handles, labels,
          ncol=1,
          frameon=False,
          loc='center left',
          bbox_to_anchor=(1, 0.5),
          fontsize=8)

plt.savefig('Se.pdf', bbox_inches='tight')
plt.savefig('Se.png', bbox_inches='tight')
plt.show()


In [ ]:
# -----------------------------------------------------------------------------
# Cell 22: Rank marker genes (scanpy.tl.rank_genes_groups)
# -----------------------------------------------------------------------------
# Purpose:
# - Identify cluster-enriched genes to support cell-type labeling.
# - Review top markers and confirm they match expected biology.

sc.tl.rank_genes_groups(adata_final, groupby='cell_type', method='wilcoxon')

top_genes_df = pd.DataFrame(adata_final.uns['rank_genes_groups']['names']).head(20)

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Copy the following and share it (e.g., in an email/message):：")
print("="*50)
print(top_genes_df)
print("="*50)

print("\nMore details for the top 5 genes ():")
sc.pl.rank_genes_groups(adata_final, n_genes=5, sharey=False, show=False)
for group in adata_final.obs['cell_type'].unique():
    print(f"\n--- {group} ---")
    df_detail = sc.get.rank_genes_groups_df(adata_final, group=group).head(5)
    print(df_detail[['names', 'logfoldchanges', 'pvals_adj']].to_string(index=False))


In [ ]:
# -----------------------------------------------------------------------------
# Cell 23: Analysis step
# -----------------------------------------------------------------------------
# Purpose:
# - Intermediate analysis step from the original notebook.
# - Consider refactoring into functions if reused.

adata_final.obsm['spatial'] = adata_final.obsm['spatial'].astype(float)

x = adata_final.obsm['spatial'][:, 0]
y = adata_final.obsm['spatial'][:, 1]

threshold = 100000

if np.any(x > threshold) or np.any(y > threshold):
    print("Detected coordinate overflow; attempting automatic fix...")

    overflow_val = 4294967296.0

    mask_x = x > threshold
    x[mask_x] = x[mask_x] - overflow_val

    mask_y = y > threshold
    y[mask_y] = y[mask_y] - overflow_val

    adata_final.obsm['spatial'][:, 0] = x
    adata_final.obsm['spatial'][:, 1] = y

    print("Fix completed！")
else:
    print("No obvious overflow detected (or already fixed)。")

adata_final.obsm['spatial'][:, 0] -= adata_final.obsm['spatial'][:, 0].min()
adata_final.obsm['spatial'][:, 1] -= adata_final.obsm['spatial'][:, 1].min()

print(f"Current X range: {adata_final.obsm['spatial'][:, 0].min()}  {adata_final.obsm['spatial'][:, 0].max()}")


In [ ]:
# -----------------------------------------------------------------------------
# Cell 24: Analysis step
# -----------------------------------------------------------------------------
# Purpose:
# - Intermediate analysis step from the original notebook.
# - Consider refactoring into functions if reused.
marker_genes_dict = {
    'Astrocyte': ['Atp1a2', 'Slc1a2'],
    'Choroid_Plexus': ['Ttr', 'Igfbp2'],
    'DA_Neuron': ['Slc6a3', 'Slc18a2'],
    'Dentate_Gyrus': ['Ncdn', 'Prox1'],
    'Excitatory_Ctx_Deep': ['Csmd1', 'Nrg3'],
    'Excitatory_Neuron': ['Nrgn', 'Slc17a7'],
    'Excitatory_Piriform': ['Lypd1', 'Nptxr'],
    'Hippocampal_Neuron': ['Hpca', 'Atp2b1'],
    'Inhibitory_Neuron': ['Gad1', 'Gad2'],
    'Microglia': ['P2ry12', 'Csf1r'],
    'Neuron_Nap1l5': ['Nap1l5', 'Peg3'],
    'Oligodendrocyte': ['Plp1', 'Mbp'],
    'Serotonergic_Neuron': ['Galntl6', 'Tph2'],
    'Thalamus_Pcp4': ['Prkcd', 'Pcp4'],
    'Unknown': ['Kcnn4', 'Iigp1']
}
marker_genes_dict2 = {
    k: [g for g in v if g in adata_final.var_names]
    for k, v in marker_genes_dict.items()
}
marker_genes_dict2 = {k:v for k,v in marker_genes_dict2.items() if len(v)>0}

celltype_order = [
    "Astrocyte","Choroid_Plexus","DA_Neuron","Dentate_Gyrus","Excitatory_Ctx_Deep",
    "Excitatory_Neuron","Excitatory_Piriform","Hippocampal_Neuron","Inhibitory_Neuron",
    "Microglia","Neuron_Nap1l5","Oligodendrocyte","Serotonergic_Neuron","Thalamus_Pcp4","Unknown"
]
celltype_order = [c for c in celltype_order if c in adata_final.obs["cell_type"].unique()]

num_genes = sum(len(v) for v in marker_genes_dict2.values())
fig_width = num_genes * 0.30 + 2.5

matplotlib.rcParams["savefig.dpi"] = 600
matplotlib.rcParams["figure.dpi"]  = 600
matplotlib.rcParams["font.family"] = "serif"
matplotlib.rcParams["font.serif"]  = ["Times New Roman"] + matplotlib.rcParams["font.serif"]

marker_list = []
for v in marker_genes_dict.values():
    marker_list.extend([g for g in v if g in adata_final.var_names])

dp = sc.pl.dotplot(
    adata_final,
    marker_list,
    groupby="cell_type",
    standard_scale="var",
    dot_max=0.8,
    dot_min=0.05,
    show=False,
    return_fig=True,
)
dp.savefig("Final_Markers_DotPlot.png")


dp.savefig("Final_Markers_DotPlot.pdf")
dp.savefig("Final_Markers_DotPlot.png")
print("✅ Figure saved：Final_Markers_DotPlot.png")


In [ ]:
# -----------------------------------------------------------------------------
# Cell 25: Analysis step
# -----------------------------------------------------------------------------
# Purpose:
# - Intermediate analysis step from the original notebook.
# - Consider refactoring into functions if reused.

fig, ax = plt.subplots(figsize=(10, 4))
x_coords = adata_final.obsm['spatial'][:, 0]

counts, bins, patches = ax.hist(x_coords, bins=200, color='#333333', alpha=0.7)
ax.set_title('Recalculated Histogram: Find the Gap Again')
ax.xaxis.set_major_locator(ticker.MaxNLocator(nbins=20))
ax.grid(True, alpha=0.5)

plt.show()


In [ ]:
# -----------------------------------------------------------------------------
# Cell 26: Spatial visualization / sanity checks
# -----------------------------------------------------------------------------
# Purpose:
# - Plot spatial distributions to confirm orientation, tissue mask, and region logic.
# - Always cross-check against histology if available.

new_split_line = 10000

adata_final.obs['condition'] = 'Control'
adata_final.obs.loc[adata_final.obsm['spatial'][:, 0] < new_split_line, 'condition'] = 'Model'

print("Group summary：")
print(adata_final.obs['condition'].value_counts())

sc.pl.spatial(
    adata_final,
    color='condition',
    spot_size=30,
    title='Fixed Coordinates Split',
    palette={'Model': '#d62728', 'Control': '#1f77b4'}
)


In [ ]:
# -----------------------------------------------------------------------------
# Cell 27: Analysis step
# -----------------------------------------------------------------------------
# Purpose:
# - Intermediate analysis step from the original notebook.
# - Consider refactoring into functions if reused.

# ==========================================
# ==========================================

if 'condition_valley' in adata_final.obs.columns:
    del adata_final.obs['condition_valley']
    print("Dropped redundant column: condition_valley")

# ==========================================
# ==========================================

cols_to_remove = ['x', 'y']
for col in cols_to_remove:
    if col in adata_final.obs.columns:
        del adata_final.obs[col]
        print(f"Dropped redundant column: {col}")

# ==========================================
# ==========================================
if 'spatial_original' in adata_final.obsm.keys():
    del adata_final.obsm['spatial_original']
    print("Removed backup matrix: spatial_original")

# ==========================================
# ==========================================
if hasattr(adata_final.obs['condition'], 'cat'):
    adata_final.obs['condition'] = adata_final.obs['condition'].cat.remove_unused_categories()

# ==========================================


In [ ]:
# -----------------------------------------------------------------------------
# Cell 30: Spatial visualization / sanity checks
# -----------------------------------------------------------------------------
# Purpose:
# - Plot spatial distributions to confirm orientation, tissue mask, and region logic.
# - Always cross-check against histology if available.

fig, ax = plt.subplots(figsize=(12, 12))

sc.pl.spatial(
    adata_final,
    color=['Th'],
    spot_size=30,
    ax=ax,
    show=False,
    title="Define ROI on the RIGHT (Control) Side"
)

ax.axis('on')
ax.xaxis.set_major_locator(ticker.MaxNLocator(nbins=30))
ax.yaxis.set_major_locator(ticker.MaxNLocator(nbins=30))
ax.grid(True, which='major', color='green', linestyle='--', alpha=0.5)

midline_x = 10000
ax.axvline(midline_x, color='black', linestyle='-', linewidth=2, label='Midline')

plt.legend()
plt.show()


In [ ]:
# -----------------------------------------------------------------------------
# Cell 31: Spatial visualization / sanity checks
# -----------------------------------------------------------------------------
# Purpose:
# - Plot spatial distributions to confirm orientation, tissue mask, and region logic.
# - Always cross-check against histology if available.
from matplotlib import patches
# ==========================================
# ==========================================
rcParams['pdf.fonttype']   = 42
rcParams['ps.fonttype']    = 42
rcParams['font.family']    = 'serif'
rcParams['font.serif']     = ['Times New Roman', 'Times', 'DejaVu Serif']
rcParams['savefig.dpi']    = 300

out_dir = 'Lateral_Cuts'
os.makedirs(out_dir, exist_ok=True)

print("="*80)
print("📊 Lateral Cuts Visualization")
print("="*80)

# ==========================================
# ==========================================
y_min, y_max = 11400, 13200

x_min_L, x_max_L = 6000, 7800
x_min_R, x_max_R = 12200, 14000

print(f"\n[Split parameters]")
print(f"  Y range: [{y_min}, {y_max}]")
print(f"  Model side (left): X ∈ [{x_min_L}, {x_max_L}]")
print(f"  Control side (right): X ∈ [{x_min_R}, {x_max_R}]")

# ==========================================
# ==========================================
mask_L = (
    (adata_final.obsm['spatial'][:, 0] >= x_min_L) &
    (adata_final.obsm['spatial'][:, 0] <= x_max_L) &
    (adata_final.obsm['spatial'][:, 1] >= y_min) &
    (adata_final.obsm['spatial'][:, 1] <= y_max)
)

mask_R = (
    (adata_final.obsm['spatial'][:, 0] >= x_min_R) &
    (adata_final.obsm['spatial'][:, 0] <= x_max_R) &
    (adata_final.obsm['spatial'][:, 1] >= y_min) &
    (adata_final.obsm['spatial'][:, 1] <= y_max)
)

mask_combined = mask_L | mask_R
adata_lateral_new = adata_final[mask_combined].copy()

adata_lateral_new.obs['split_group'] = 'Unknown'
adata_lateral_new.obs.loc[adata_lateral_new.obsm['spatial'][:, 0] < 10000, 'split_group'] = 'Model_Lateral'
adata_lateral_new.obs.loc[adata_lateral_new.obsm['spatial'][:, 0] > 10000, 'split_group'] = 'Control_Lateral'

print(f"\n[Split summary]")
print(f"  Model siden_units: {mask_L.sum():,}")
print(f"  Control siden_units: {mask_R.sum():,}")
print(f"  n_units: {mask_combined.sum():,}")

# ==========================================
# ==========================================
print(f"\n【saved】")

fig, ax = plt.subplots(figsize=(10, 8), facecolor='white')

sc.pl.spatial(adata_final, color=None, spot_size=20, alpha=0.1, ax=ax, show=False,
              title="Updated Lateral Cuts (6000-7800 vs 12200-14000)")
ax.collections[0].set_color('gray')

rect_L = patches.Rectangle((x_min_L, y_min), x_max_L-x_min_L, y_max-y_min,
                           linewidth=2.5, edgecolor='red', facecolor='none',
                           label='Model (Lateral)')
ax.add_patch(rect_L)

rect_R = patches.Rectangle((x_min_R, y_min), x_max_R-x_min_R, y_max-y_min,
                           linewidth=2.5, edgecolor='blue', facecolor='none',
                           label='Control (Lateral)')
ax.add_patch(rect_R)

ax.axvline(10000, color='black', linestyle='--', linewidth=1.5, alpha=0.6)

ax.set_title("Updated Lateral Cuts (6000-7800 vs 12200-14000)",
             fontsize=13, fontweight='bold', family='serif', pad=15)

ax.set_xlabel("X Coordinate (µm)", fontsize=11, fontweight='bold', family='serif')
ax.set_ylabel("Y Coordinate (µm)", fontsize=11, fontweight='bold', family='serif')

ax.tick_params(labelsize=10)
for label in ax.get_xticklabels() + ax.get_yticklabels():
    label.set_fontfamily('serif')

ax.axis('on')
ax.grid(True, linestyle='--', alpha=0.3, linewidth=0.5)

legend = ax.legend(loc='upper right', fontsize=10, frameon=True,
                   fancybox=True, shadow=True)
legend.get_frame().set_facecolor('white')
legend.get_frame().set_alpha(0.9)
for text in legend.get_texts():
    text.set_fontfamily('serif')

plt.tight_layout()

pdf_file = os.path.join(out_dir, 'Lateral_Cuts_Check.pdf')
fig.savefig(pdf_file, dpi=300, bbox_inches='tight', transparent=True)

png_file = os.path.join(out_dir, 'Lateral_Cuts_Check.png')
fig.savefig(png_file, dpi=600, bbox_inches='tight', facecolor='white')

print(f"\n✅ figure:")
print(f"   - PDF (): {pdf_file}")
print(f"   - PNG (600 DPI): {png_file}")

plt.show()


stats_data = {
    'Region': ['Model (Left)', 'Control (Right)', 'Total'],
    'X_Range': [f'[{x_min_L}, {x_max_L}]', f'[{x_min_R}, {x_max_R}]', '-'],
    'Y_Range': [f'[{y_min}, {y_max}]', f'[{y_min}, {y_max}]', '-'],
    'Cell_Count': [mask_L.sum(), mask_R.sum(), mask_combined.sum()],
    'Width': [x_max_L - x_min_L, x_max_R - x_min_R, '-'],
    'Height': [y_max - y_min, y_max - y_min, '-']
}

stats_df = pd.DataFrame(stats_data)
stats_file = os.path.join(out_dir, 'Lateral_Cuts_Statistics.xlsx')
stats_df.to_excel(stats_file, index=False)

print(f"   - : {stats_file}")
print("="*80)


In [ ]:
# -----------------------------------------------------------------------------
# Cell 32: Spatial visualization / sanity checks
# -----------------------------------------------------------------------------
# Purpose:
# - Plot spatial distributions to confirm orientation, tissue mask, and region logic.
# - Always cross-check against histology if available.

# ==========================================
# ==========================================
rcParams['pdf.fonttype']   = 42
rcParams['ps.fonttype']    = 42
rcParams['font.family']    = 'serif'
rcParams['font.serif']     = ['Times New Roman', 'Times', 'DejaVu Serif']
rcParams['savefig.dpi']    = 300

out_dir = 'Stitched_Spatial_Plots'
os.makedirs(out_dir, exist_ok=True)

print("="*80)
print("📊 Lateral Cuts Stitched Spatial Plot")
print("="*80)

adata_stitch = adata_lateral_new.copy()


adata_stitch.obs['cell_type'] = adata_stitch.obs['cell_type'].astype(str)
adata_stitch.obs.loc[adata_stitch.obs['cell_type'] == 'Immune_Peripheral', 'cell_type'] = 'Unknown'
adata_stitch.obs['cell_type'] = adata_stitch.obs['cell_type'].astype('category')

adata_stitch.obs['cell_type'] = adata_stitch.obs['cell_type'].cat.set_categories(
    adata_final.obs['cell_type'].cat.categories
)

adata_stitch.uns['cell_type_colors'] = adata_final.uns['cell_type_colors'].copy()
# ============================================================


xy = adata_stitch.obsm['spatial'].copy()

y_min, y_max = 11400, 13200
x_min_L, x_max_L = 6000, 7800
x_min_R, x_max_R = 12200, 14000

gap = 300

print(f"\n【】")
print(f"  Y range: [{y_min}, {y_max}]")
print(f"  left X range: [{x_min_L}, {x_max_L}] (: {x_max_L - x_min_L})")
print(f"  right X range: [{x_min_R}, {x_max_R}] (: {x_max_R - x_min_R})")
print(f"  : {gap} units")

mask_left  = xy[:, 0] < 10000
mask_right = xy[:, 0] > 10000

print(f"\n【n_units】")
print(f"  left (Model): {mask_left.sum():,} ")
print(f"  right (Control): {mask_right.sum():,} ")
print(f"  : {len(xy):,} ")

xy[:, 1] = xy[:, 1] - y_min

xy[mask_left, 0] = xy[mask_left, 0] - x_min_L

left_width = (x_max_L - x_min_L)
xy[mask_right, 0] = (xy[mask_right, 0] - x_min_R) + left_width + gap

adata_stitch.obsm['spatial'] = xy

print(f"\n【saved】")

fig, ax = plt.subplots(figsize=(6.5, 4.5), facecolor='white')
ax.set_facecolor('white')

sc.pl.spatial(
    adata_stitch,
    color='cell_type',
    spot_size=35,
    alpha=0.95,
    frameon=False,
    legend_loc='right margin',
    legend_fontsize=9,
    title='',
    ax=ax,
    show=False
)


for spine in ax.spines.values():
    spine.set_visible(False)

ax.set_xticks([])
ax.set_yticks([])
ax.set_axis_off()

legend = ax.get_legend()
if legend:
    legend.set_frame_on(False)
    legend.set_title('', prop={'size': 10, 'weight': 'bold', 'family': 'serif'})
    for text in legend.get_texts():
        text.set_fontfamily('serif')
        text.set_fontsize(9)


plt.tight_layout()

pdf_file = os.path.join(out_dir, "LateralCuts_celltype_stitched.pdf")
png_file = os.path.join(out_dir, "LateralCuts_celltype_stitched.png")

fig.savefig(pdf_file, bbox_inches="tight", transparent=True)
fig.savefig(png_file, dpi=600, bbox_inches="tight", facecolor='white')

plt.show()

print(f"\n✅ :")
print(f"   - PDF (): {pdf_file}")
print(f"   - PNG (600 DPI): {png_file}")
print("="*80)


In [ ]:
# -----------------------------------------------------------------------------
# Cell 33: Analysis step
# -----------------------------------------------------------------------------
# Purpose:
# - Intermediate analysis step from the original notebook.
# - Consider refactoring into functions if reused.

# ==========================================
# ==========================================
target_cells = ['DA_Neuron', 'Microglia', 'Astrocyte', 'Oligodendrocyte']

counts = pd.crosstab(adata_lateral_new.obs['cell_type'], adata_lateral_new.obs['split_group'])
res_counts = counts.loc[counts.index.intersection(target_cells)]

cols = ['Control_Lateral', 'Model_Lateral']
for c in cols:
    if c not in res_counts.columns: res_counts[c] = 0
res_counts = res_counts[cols]

print("\n" + "="*60)
print("  📊  (Wider Lateral) ")
print(f"  Model: {x_min_L}-{x_max_L} | Control: {x_min_R}-{x_max_R}")
print("="*60)
print(f"{'Cell Type':<20} {'Ctrl(Lat)':<10} {'Mod(Lat)':<10} {'Diff':<8} {'Loss/Change':<10}")
print("-" * 65)

for cell in target_cells:
    if cell in res_counts.index:
        c = res_counts.loc[cell, 'Control_Lateral']
        m = res_counts.loc[cell, 'Model_Lateral']
        diff = m - c

        if c > 0:
            pct = (diff / c) * 100
        else:
            pct = 0

        mark = ""
        if cell == 'DA_Neuron' and pct < -20: mark = "📉 (Loss)"
        if (cell == 'Microglia' or cell == 'Astrocyte') and pct > 20: mark = "🔥 (Inflam)"

        print(f"{cell:<20} {c:<10} {m:<10} {diff:<8} {pct:+.2f}% {mark}")

# ==========================================
# ==========================================
fig, ax = plt.subplots(figsize=(8, 5))
res_counts.plot(kind='bar', color=['#1f77b4', '#d62728'], ax=ax, rot=0, edgecolor='black')
ax.set_title('Cell Counts Comparison (Updated Coordinates)')
ax.set_ylabel('Number of Cells')
ax.legend(title='Region')
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# -----------------------------------------------------------------------------
# Cell 34: Composition statistics (Fisher / Chi-square)
# -----------------------------------------------------------------------------
# Purpose:
# - Test whether cell-type composition differs between groups/regions.
# - Use Fisher's exact test for 2x2; use Chi-square for larger contingency tables.

# ==========================================
# ==========================================
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial', 'DejaVu Sans']
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

# ==========================================
# ==========================================
cells_bottom_to_top = ['Oligodendrocyte', 'Astrocyte', 'Microglia', 'DA_Neuron']
cell_col  = 'cell_type'
group_col = 'split_group'
model_key = 'Model_Lateral'
ctrl_key  = 'Control_Lateral'

USE_FDR_FOR_STARS = True
SHOW_NS = False

def bh_fdr(pvals):
    pvals = np.asarray(pvals, dtype=float)
    n = len(pvals)
    order = np.argsort(pvals)
    ranked = pvals[order]
    q = ranked * n / (np.arange(1, n+1))
    q = np.minimum.accumulate(q[::-1])[::-1]
    q = np.clip(q, 0, 1)
    out = np.empty_like(q)
    out[order] = q
    return out

def p_to_star(p):
    if p < 0.001: return '***'
    if p < 0.01:  return '**'
    if p < 0.05:  return '*'
    return 'ns'

# ==========================================
# ==========================================
grp = adata_lateral_new.obs[group_col].astype(str).str.strip()
ct  = adata_lateral_new.obs[cell_col].astype(str)

counts = pd.crosstab(ct, grp)

total_model_all = int(counts[model_key].sum()) if model_key in counts.columns else 0
total_ctrl_all  = int(counts[ctrl_key].sum())  if ctrl_key  in counts.columns else 0

df = counts.reindex(cells_bottom_to_top).copy()
for k in [model_key, ctrl_key]:
    if k not in df.columns:
        df[k] = 0
df = df[[model_key, ctrl_key]].fillna(0).astype(int)

row_total = (df[model_key] + df[ctrl_key]).replace(0, np.nan)
pct_model = (df[model_key] / row_total * 100).fillna(0)
pct_ctrl  = (df[ctrl_key]  / row_total * 100).fillna(0)

# ==========================================
# ==========================================
pvals = []
test_used = []

for cell in df.index:
    n_model = int(df.loc[cell, model_key])
    n_ctrl  = int(df.loc[cell, ctrl_key])

    obs = np.array([
        [n_model, n_ctrl],
        [total_model_all - n_model, total_ctrl_all - n_ctrl]
    ], dtype=int)

    chi2, p_chi, dof, exp = chi2_contingency(obs, correction=False)

    if (exp < 5).any():
        # Fisher exact (two-sided)
        _, p = fisher_exact(obs, alternative='two-sided')
        test_used.append('Fisher')
    else:
        p = p_chi
        test_used.append('Chi2')

    pvals.append(p)

pvals = np.array(pvals, dtype=float)
qvals = bh_fdr(pvals)

stars = []
for p, q in zip(pvals, qvals):
    s = p_to_star(q if USE_FDR_FOR_STARS else p)
    if (not SHOW_NS) and (s == 'ns'):
        s = ''
    stars.append(s)

# ==========================================
# ==========================================
df_plot = pd.DataFrame({
    'Cell': df.index,
    'Model': df[model_key].values,
    'Control': df[ctrl_key].values,
    'Pct_Model': pct_model.values,
    'Pct_Control': pct_ctrl.values,
    'p': pvals,
    'q': qvals,
    'test': test_used,
    'star': stars
}).reset_index(drop=True)

df_plot['Label'] = df_plot['Cell'] + df_plot['star']

# ==========================================
# ==========================================
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif']  = ['Times New Roman'] + matplotlib.rcParams['font.serif']
print(matplotlib.rcParams['font.serif'])
fig, ax = plt.subplots(figsize=(8, 4))
y_pos = np.arange(len(df_plot))
height = 0.85

color_model = '#990000'
color_ctrl  = '#00008B'

ax.barh(y_pos, df_plot['Pct_Model'], height,
        color=color_model, edgecolor='white', linewidth=1)
ax.barh(y_pos, df_plot['Pct_Control'], height,
        left=df_plot['Pct_Model'],
        color=color_ctrl, edgecolor='white', linewidth=1)

for yi, row in enumerate(df_plot.itertuples(index=False)):
    if row.Pct_Model > 0:
        ax.text(row.Pct_Model/2, yi, f"{int(row.Model)}",
                ha='center', va='center', color='white', fontsize=14, fontweight='bold')
    if row.Pct_Control > 0:
        ax.text(row.Pct_Model + row.Pct_Control/2, yi, f"{int(row.Control)}",
                ha='center', va='center', color='white', fontsize=14, fontweight='bold')

den = total_model_all + total_ctrl_all
global_model_pct = (total_model_all / den * 100) if den > 0 else 0
ax.axvline(global_model_pct, color='black', linestyle='--', linewidth=1.5, ymin=-0.05, ymax=1.05)
ax.text(global_model_pct, -0.6, f"{global_model_pct:.2f}",
        ha='center', va='top', fontsize=12, rotation=90)

ax.set_yticks(y_pos)
ax.set_yticklabels(df_plot['Label'], fontsize=14, color='black')
ax.tick_params(axis='y', length=0)

ax.set_xlim(0, 100)
ax.set_xticks([0, 100])
ax.set_xticklabels(['0.00', '100.00'], fontsize=12, rotation=90)
ax.set_xlabel('Origin %', fontsize=14)

for sp in ['top', 'right', 'bottom', 'left']:
    ax.spines[sp].set_visible(True)

legend_elements = [
    Patch(facecolor=color_model, label="Model"),
    Patch(facecolor=color_ctrl,  label="Control"),
]
ax.legend(handles=legend_elements, loc='lower center', bbox_to_anchor=(0.5, 1.02),
          ncol=2, frameon=False, fontsize=12, handlelength=1.0)

plt.tight_layout()
plt.savefig('Cell_Proportion_Stacked_withStars.pdf', bbox_inches='tight', dpi=600)
plt.savefig('Cell_Proportion_Stacked_withStars.png', bbox_inches='tight', dpi=600)
plt.show()

print(df_plot[['Cell','Model','Control','Pct_Model','Pct_Control','test','p','q','star']])


In [ ]:
# -----------------------------------------------------------------------------
# Cell 35: Analysis step
# -----------------------------------------------------------------------------
# Purpose:
# - Intermediate analysis step from the original notebook.
# - Consider refactoring into functions if reused.

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial', 'DejaVu Sans']
plt.rcParams['pdf.fonttype'] = 42

target_cells = ['Oligodendrocyte', 'Microglia', 'DA_Neuron', 'Astrocyte']
counts = pd.crosstab(adata_lateral_new.obs['split_group'], adata_lateral_new.obs['cell_type'])

df_counts = counts[target_cells].copy()
df_counts = df_counts.reindex(['Control_Lateral', 'Model_Lateral'])

df_props = df_counts.div(df_counts.sum(axis=1), axis=0)

colors_map = {
    'Astrocyte':       '#F4A582',  # 粉橙
    'Microglia':       '#66C2A5',  # 青绿
    'Oligodendrocyte': '#4575B4',  # 深蓝
    'DA_Neuron':       '#D73027'   # 深红 (重点)
}
plot_colors = [colors_map[c] for c in df_props.columns]

matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif']  = ['Times New Roman'] + matplotlib.rcParams['font.serif']
fig, ax = plt.subplots(figsize=(5, 6))

df_props.plot(kind='bar', stacked=True, color=plot_colors, ax=ax, width=0.85, edgecolor='white', linewidth=0.5)

ax.grid(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

ax.set_xticklabels(['Control', 'Model'], rotation=0, fontsize=14)
ax.set_xlabel('')
ax.set_ylim(0, 1)
ax.set_ylabel('Proportion', fontsize=14)
ax.set_yticks([0, 0.25, 0.5, 0.75, 1.0])

handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels), title='Cell Type',
          loc='center left', bbox_to_anchor=(1, 0.5), frameon=False)


plt.tight_layout()
plt.savefig('Cell_Composition_4Types_NoGrid.pdf', bbox_inches='tight')
plt.savefig('Cell_Composition_4Types_NoGrid.png', bbox_inches='tight', dpi=600)
plt.show()


In [ ]:
# -----------------------------------------------------------------------------
# Cell 36: Analysis step
# -----------------------------------------------------------------------------
# Purpose:
# - Intermediate analysis step from the original notebook.
# - Consider refactoring into functions if reused.

counts_all = pd.crosstab(adata_lateral_new.obs['split_group'], adata_lateral_new.obs['cell_type'])

counts_all = counts_all.reindex(['Control_Lateral', 'Model_Lateral'])

df_props_all = counts_all.div(counts_all.sum(axis=1), axis=0)

num_cells = len(df_props_all.columns)
colors_palette = sns.color_palette("tab20", n_colors=num_cells)

matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif']  = ['Times New Roman'] + matplotlib.rcParams['font.serif']
fig, ax = plt.subplots(figsize=(6, 7))

df_props_all.plot(kind='bar', stacked=True, color=colors_palette, ax=ax, width=0.85, edgecolor='white', linewidth=0.3)

ax.grid(False)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(True)

ax.set_xticklabels(['Control', 'Model'], rotation=0, fontsize=14)
ax.set_xlabel('')
ax.set_ylim(0, 1)
ax.set_ylabel('Proportion (All Cells)', fontsize=14)

handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels), title='Cell Types',
          loc='upper left', bbox_to_anchor=(1, 1), frameon=False, fontsize=10)

plt.tight_layout()
plt.savefig('Cell_Composition_AllCells.pdf', bbox_inches='tight')
plt.savefig('Cell_Composition_AllCells.png', bbox_inches='tight', dpi=600)
print("✅ saved！")
plt.show()


In [ ]:
# -----------------------------------------------------------------------------
# Cell 37: Gene-set scoring (signature scores)
# -----------------------------------------------------------------------------
# Purpose:
# - Compute simple signature scores (e.g., CD8 cytotoxicity, IFN response).
# - Interpret scores cautiously: they summarize gene set expression, not pathway activity.

# ==========================================
# ==========================================
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif']  = ['Times New Roman'] + matplotlib.rcParams['font.serif']
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

# ==========================================
# ==========================================
target_scores = {
    'Microglia': {
        'score_name': 'DAM_Score',
        'genes': ['Trem2', 'Tyrobp', 'Ctsb', 'Apoe', 'Lpl', 'Cst7']
    },
    'Astrocyte': {
        'score_name': 'Reactivity_Score',
        'genes': ['Gfap', 'Vim', 'Serpina3n', 'Lcn2', 'C3']
    },
    'Oligodendrocyte': {
        'score_name': 'Metabolic_Stress',
        'genes': ['Hspa5', 'Ddit3', 'Xbp1', 'Hsp90b1']
    }
}

plot_order = ['Microglia', 'Astrocyte', 'Oligodendrocyte']

# ==========================================
# ==========================================
adata_score = adata_lateral_new.copy()
plot_data_list = []
p_values = {}

print("...")

for cell_type in plot_order:
    info = target_scores[cell_type]
    score_key = info['score_name']
    genes = info['genes']

    subset = adata_score[adata_score.obs['cell_type'] == cell_type].copy()

    valid_genes = [g for g in genes if g in subset.var_names]

    sc.tl.score_genes(subset, gene_list=valid_genes, score_name=score_key, ctrl_size=50)

    df_temp = pd.DataFrame({
        'Score': subset.obs[score_key].values,
        'Group': subset.obs['split_group'].map({'Control_Lateral': 'Control', 'Model_Lateral': 'Model'}),
        'Cell_Type': cell_type,
        'Score_Name': score_key
    })
    plot_data_list.append(df_temp)

    vals_c = df_temp[df_temp['Group'] == 'Control']['Score']
    vals_m = df_temp[df_temp['Group'] == 'Model']['Score']
    stat, p = ranksums(vals_m, vals_c)
    p_values[cell_type] = p

    print(f"  > {cell_type} ({score_key}): P-value = {p:.2e}")

df_viz = pd.concat(plot_data_list)

# ==========================================
# ==========================================
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(4, 8), sharex=True)
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif']  = ['Times New Roman'] + matplotlib.rcParams['font.serif']
cell_colors = {
    "Microglia": "#2F3E73",        # 蓝
    "Astrocyte": "#86CDBF",        # 青/薄荷绿
    "Oligodendrocyte": "#E58C74"   # 棕红/鲑红
}

for i, cell_type in enumerate(plot_order):
    ax = axes[i]
    data = df_viz[df_viz['Cell_Type'] == cell_type]
    score_name = target_scores[cell_type]['score_name']
    c = cell_colors[cell_type]

    sns.violinplot(
        data=data, x='Group', y='Score',
        color=c, ax=ax,
        inner=None,
        linewidth=0.8,
        saturation=1.0,
        width=0.8,
        cut=0
    )

    ax.grid(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(True)

    ax.tick_params(axis='y', length=0, width=0)
    ax.tick_params(axis='x', length=0)

    ax.set_ylabel(score_name, fontsize=11, labelpad=10)
    ax.set_xlabel('')

    p = p_values[cell_type]
    sig_star = "***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else "ns"
    ax.text(
        1.02, 0.5, f"{cell_type}\n({sig_star})",
        transform=ax.transAxes, ha='left', va='center',
        fontsize=12, fontweight='bold', color='black'
    )

axes[-1].set_xticklabels(['Control', 'Model'], fontsize=12, fontweight='bold')
axes[-1].spines['bottom'].set_visible(True)
axes[-1].tick_params(axis='x', length=5)

plt.subplots_adjust(hspace=0.1)

plt.savefig('Stacked_Violin_Scores_Strict.pdf', bbox_inches='tight')
plt.savefig('Stacked_Violin_Scores_Strict.png', bbox_inches='tight', dpi=600)
print("\n✅ Figure saved：Stacked_Violin_Scores_Strict.png")
plt.show()


In [ ]:
# -----------------------------------------------------------------------------
# Cell 38: Expression statistics (MWU / rank-based)
# -----------------------------------------------------------------------------
# Purpose:
# - Compare expression distributions between groups using non-parametric tests.
# - Report effect sizes and adjust p-values if many genes are tested.

# ==========================================
# ==========================================
t_cell_genes = ['Cd3d', 'Cd3e', 'Cd3g', 'Cd8a', 'Cd4', 'Cd28']

valid_t_genes = [g for g in t_cell_genes if g in adata_lateral_new.var_names]
if not valid_t_genes:
    valid_t_genes = [g.capitalize() for g in t_cell_genes if g.capitalize() in adata_lateral_new.var_names]

print(f" T  Marker: {valid_t_genes}")

def get_t_cell_coords(adata_subset):
    """ T """
    if not valid_t_genes: return np.array([])

    try:
        expr = adata_subset[:, valid_t_genes].X.toarray()
    except:
        expr = adata_subset[:, valid_t_genes].X

    expr_sum = expr.sum(axis=1)
    mask = expr_sum > 0
    return adata_subset.obsm['spatial'][mask]

# ==========================================
# ==========================================

def analyze_spatial_relations(adata, group_col, group_name):
    subset = adata[adata.obs[group_col] == group_name]

    coords_microglia = subset[subset.obs['cell_type'] == 'Microglia'].obsm['spatial']
    coords_da = subset[subset.obs['cell_type'] == 'DA_Neuron'].obsm['spatial']
    coords_t_cell = get_t_cell_coords(subset)

    results = {}

    if len(coords_t_cell) > 0 and len(coords_microglia) > 0:
        tree_mg = KDTree(coords_microglia)

        radii = [100, 200, 400]
        density_res = {}

        for r in radii:
            counts = tree_mg.query_ball_point(coords_t_cell, r)
            counts_len = [len(c) for c in counts]
            density_res[r] = counts_len

        results['T_Microglia_Density'] = density_res

        tree_t = KDTree(coords_t_cell)
        dist_mg_to_t, _ = tree_t.query(coords_microglia, k=1)
        results['Dist_MG_to_T'] = dist_mg_to_t

    else:
        results['T_Microglia_Density'] = None
        results['Dist_MG_to_T'] = None

    if len(coords_da) > 0 and len(coords_microglia) > 0:
        tree_mg = KDTree(coords_microglia)
        dist_da_to_mg, _ = tree_mg.query(coords_da, k=1)
        results['Dist_DA_to_MG'] = dist_da_to_mg
    else:
        results['Dist_DA_to_MG'] = None

    return results

# ==========================================
# ==========================================
print(" ()...")
res_model = analyze_spatial_relations(adata_lateral_new, 'split_group', 'Model_Lateral')
res_ctrl = analyze_spatial_relations(adata_lateral_new, 'split_group', 'Control_Lateral')

# ==========================================
# ==========================================
fig, axs = plt.subplots(1, 3, figsize=(18, 6))

r_focus = 200
data_density = []
if res_model['T_Microglia_Density'] and res_ctrl['T_Microglia_Density']:
    data_density.extend([{'Count': c, 'Group': 'Control'} for c in res_ctrl['T_Microglia_Density'][r_focus]])
    data_density.extend([{'Count': c, 'Group': 'Model'} for c in res_model['T_Microglia_Density'][r_focus]])

    df_dens = pd.DataFrame(data_density)
    sns.boxplot(data=df_dens, x='Group', y='Count', ax=axs[0], palette=['blue', 'red'], showfliers=False)

    s, p = ranksums(df_dens[df_dens['Group']=='Model']['Count'], df_dens[df_dens['Group']=='Control']['Count'])
    sig = "***" if p<0.001 else ("*" if p<0.05 else "ns")
    axs[0].set_title(f"Microglia count around CD3+ Spots\n(Radius={r_focus}, P={p:.2e} {sig})")
    axs[0].set_ylabel('Number of Microglia')
else:
    axs[0].text(0.5, 0.5, "Insufficient T-cell signals", ha='center')

data_dist_t = []
if res_model['Dist_MG_to_T'] is not None and res_ctrl['Dist_MG_to_T'] is not None:
    data_dist_t.extend([{'Dist': d, 'Group': 'Control'} for d in res_ctrl['Dist_MG_to_T']])
    data_dist_t.extend([{'Dist': d, 'Group': 'Model'} for d in res_model['Dist_MG_to_T']])

    df_dt = pd.DataFrame(data_dist_t)
    sns.violinplot(data=df_dt, x='Group', y='Dist', ax=axs[1], palette=['blue', 'red'])

    s, p = ranksums(df_dt[df_dt['Group']=='Model']['Dist'], df_dt[df_dt['Group']=='Control']['Dist'])
    sig = "***" if p<0.001 else ("*" if p<0.05 else "ns")
    axs[1].set_title(f"Distance: Microglia -> Nearest CD3+\n(Lower = Attraction, P={p:.2e} {sig})")
    axs[1].set_ylabel('Distance (coords)')
    axs[1].set_ylim(0, 2000)

data_dist_da = []
if res_model['Dist_DA_to_MG'] is not None and res_ctrl['Dist_DA_to_MG'] is not None:
    data_dist_da.extend([{'Dist': d, 'Group': 'Control'} for d in res_ctrl['Dist_DA_to_MG']])
    data_dist_da.extend([{'Dist': d, 'Group': 'Model'} for d in res_model['Dist_DA_to_MG']])

    df_da = pd.DataFrame(data_dist_da)
    sns.violinplot(data=df_da, x='Group', y='Dist', ax=axs[2], palette=['blue', 'red'])

    s, p = ranksums(df_da[df_da['Group']=='Model']['Dist'], df_da[df_da['Group']=='Control']['Dist'])
    sig = "***" if p<0.001 else ("*" if p<0.05 else "ns")
    axs[2].set_title(f"Distance: DA Neuron -> Nearest Microglia\n(Lower = Neurophagia, P={p:.2e} {sig})")
    axs[2].set_ylabel('Distance (coords)')
    axs[2].set_ylim(0, 1000)

plt.tight_layout()
plt.show()

# ==========================================
# ==========================================
print("\n" + "="*60)
print("  📏 T  ()")
print("="*60)
if res_model['T_Microglia_Density']:
    print(f"{'Radius':<10} {'Ctrl_Avg':<10} {'Model_Avg':<10} {'Diff':<10} {'P-value'}")
    for r in [100, 200, 400]:
        c_vals = res_ctrl['T_Microglia_Density'][r]
        m_vals = res_model['T_Microglia_Density'][r]

        c_mean = np.mean(c_vals) if len(c_vals)>0 else 0
        m_mean = np.mean(m_vals) if len(m_vals)>0 else 0

        if len(c_vals)>0 and len(m_vals)>0:
            s, p = ranksums(m_vals, c_vals)
        else:
            p = 1.0

        print(f"{r:<10} {c_mean:.2f}      {m_mean:.2f}      {m_mean-c_mean:+.2f}      {p:.2e}")
else:
    print(" T 。")


In [ ]:
# -----------------------------------------------------------------------------
# Cell 39: Gene-set scoring (signature scores)
# -----------------------------------------------------------------------------
# Purpose:
# - Compute simple signature scores (e.g., CD8 cytotoxicity, IFN response).
# - Interpret scores cautiously: they summarize gene set expression, not pathway activity.

# ==========================================
# ==========================================
imm = adata_lateral_new.copy()

CD8_SIG = ["Trac","Cd3d","Cd3e","Cd8a","Cd8b1","Nkg7","Gzmb","Prf1","Ccl5"]
IFN_SIG = ["Isg15","Ifit1","Ifit3","Oasl2","Rsad2","Usp18","Irf7","Stat1"]

cd8_genes = [g for g in CD8_SIG if g in imm.var_names]
if not cd8_genes: cd8_genes = [g.capitalize() for g in CD8_SIG if g.capitalize() in imm.var_names]

ifn_genes = [g for g in IFN_SIG if g in imm.var_names]
if not ifn_genes: ifn_genes = [g.capitalize() for g in IFN_SIG if g.capitalize() in imm.var_names]

print(f" CD8 : {len(cd8_genes)} ")
print(f" IFN : {len(ifn_genes)} ")

# ==========================================
# ==========================================
sc.tl.score_genes(imm, gene_list=cd8_genes, score_name="score_cd8", ctrl_size=50)
sc.tl.score_genes(imm, gene_list=ifn_genes, score_name="score_ifn", ctrl_size=50)

# ==========================================
# ==========================================
q_cd8 = imm.obs["score_cd8"].quantile(0.80)
q_ifn = imm.obs["score_ifn"].quantile(0.80)

print(f"Cutoff CD8 (80th): {q_cd8:.3f}")
print(f"Cutoff IFN (80th): {q_ifn:.3f}")

def gate(row):
    cd8 = row["score_cd8"] >= q_cd8
    ifn = row["score_ifn"] >= q_ifn

    if cd8 and ifn:
        return "Double_high"
    elif cd8 and (not ifn):
        return "CD8_like"
    elif ifn and (not cd8):
        return "IFN_like"
    else:
        return "Ambiguous"

imm.obs["immune_subtype"] = imm.obs.apply(gate, axis=1)

print("\n:")
print(imm.obs["immune_subtype"].value_counts())

# ==========================================
# ==========================================
plt.figure(figsize=(7, 6))
sns.scatterplot(
    data=imm.obs.sample(frac=1, random_state=42),
    x="score_cd8", y="score_ifn",
    hue="immune_subtype",
    palette={
        "Double_high": "#d62728", # 红 (双高)
        "CD8_like": "#1f77b4",    # 蓝 (T细胞)
        "IFN_like": "#ff7f0e",    # 橙 (炎症)
        "Ambiguous": "#e0e0e0"    # 灰
    },
    s=15, alpha=0.6, linewidth=0
)
plt.axvline(q_cd8, color="black", linestyle="--")
plt.axhline(q_ifn, color="black", linestyle="--")
plt.title("Immune Subtype Gating Strategy")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()
plt.show()


In [ ]:
# -----------------------------------------------------------------------------
# Cell 40: Analysis step
# -----------------------------------------------------------------------------
# Purpose:
# - Intermediate analysis step from the original notebook.
# - Consider refactoring into functions if reused.

# ==========================================
# ==========================================
t_broad_genes = [
    'Cd3d', 'Cd3e', 'Cd3g', # CD3 co-receptors
    'Cd4',                  # Helper T
    'Cd8a', 'Cd8b1',        # Cytotoxic T
    'Trac', 'Trbc1', 'Trbc2'# TCR constant regions
]

adata_viz = adata_lateral_new.copy()

valid_t_genes = []
for g in t_broad_genes:
    if g in adata_viz.var_names:
        valid_t_genes.append(g)
    elif g.capitalize() in adata_viz.var_names:
        valid_t_genes.append(g.capitalize())
valid_t_genes = sorted(list(set(valid_t_genes)))

print(f"✅  T  ({len(valid_t_genes)}):")
print(valid_t_genes)

# ==========================================
# ==========================================
def get_layer_data(adata, group_name):
    """"""
    subset = adata[adata.obs['split_group'] == group_name]

    coords_bg = subset.obsm['spatial']

    coords_mg = subset[subset.obs['cell_type'] == 'Microglia'].obsm['spatial']

    if len(valid_t_genes) > 0:
        try:
            t_sum = subset[:, valid_t_genes].X.sum(axis=1).A1
        except:
            t_sum = subset[:, valid_t_genes].X.sum(axis=1)

        is_t_signal = t_sum > 0.1
        coords_t = subset.obsm['spatial'][is_t_signal]
        t_count = len(coords_t)
    else:
        coords_t = np.array([])
        t_count = 0

    return coords_bg, coords_mg, coords_t, t_count

# ==========================================
# ==========================================
fig, axs = plt.subplots(1, 2, figsize=(18, 9))
groups = ['Control_Lateral', 'Model_Lateral']
titles = ['Control Side (Right)', 'Model Side (Left)']

style_bg = {'c': '#e0e0e0', 's': 15, 'alpha': 0.3, 'label': 'Tissue Background'}
style_mg = {'c': '#9467bd', 's': 40, 'alpha': 0.4, 'label': 'Microglia (Background)'} # 紫色云团
style_t  = {'c': '#d62728', 's': 120, 'marker': '*', 'linewidth': 0.5, 'edgecolor':'k', 'label': 'T-Cell Signal (Gene+)'} # 红色星星

for i, group in enumerate(groups):
    ax = axs[i]
    c_bg, c_mg, c_t, n_t = get_layer_data(adata_viz, group)

    ax.scatter(c_bg[:, 0], c_bg[:, 1], **style_bg)

    ax.scatter(c_mg[:, 0], c_mg[:, 1], **style_mg)

    if n_t > 0:
        ax.scatter(c_t[:, 0], c_t[:, 1], **style_t)
        ax.scatter(c_t[:, 0], c_t[:, 1], s=250, facecolors='none', edgecolors='#d62728', alpha=0.4)

    ax.set_title(f"{titles[i]}\nDetected T-Signals: {n_t}", fontsize=14, fontweight='bold')
    ax.set_aspect('equal')
    ax.axis('off')

    if i == 1:
        legend_elements = [
            Line2D([0], [0], marker='o', color='w', markerfacecolor=style_mg['c'], markersize=10, alpha=style_mg['alpha'], label=style_mg['label']),
            Line2D([0], [0], marker='*', color='w', markerfacecolor=style_t['c'], markersize=15, markeredgecolor='k', label=style_t['label'])
        ]
        ax.legend(handles=legend_elements, loc='upper right', fontsize=11)

plt.tight_layout()
plt.show()


In [ ]:
# -----------------------------------------------------------------------------
# Cell 41: Expression statistics (MWU / rank-based)
# -----------------------------------------------------------------------------
# Purpose:
# - Compare expression distributions between groups using non-parametric tests.
# - Report effect sizes and adjust p-values if many genes are tested.

# ==========================================
# ==========================================
adata_analyze = adata_lateral_new.copy()

gene_target = 'Trac'
if gene_target not in adata_analyze.var_names:
    if gene_target.upper() in adata_analyze.var_names:
        gene_target = gene_target.upper()
    else:
        print(f"⚠️  {gene_target} ，。")
        if 'Cd3e' in adata_analyze.var_names:
            print("⚠️  Cd3e ...")
            gene_target = 'Cd3e'

print(f"🎯 : {gene_target}")

# ==========================================
# ==========================================
def analyze_trac_microglia_relation(adata, group_name, radius=150):
    subset = adata[adata.obs['split_group'] == group_name]

    try:
        expr = subset[:, gene_target].X.toarray().flatten()
    except:
        expr = subset[:, gene_target].X.flatten()

    trac_coords = subset.obsm['spatial'][expr > 0]

    mg_coords = subset[subset.obs['cell_type'] == 'Microglia'].obsm['spatial']

    results = {
        'dist_to_mg': [],
        'mg_count': []
    }

    if len(trac_coords) == 0 or len(mg_coords) == 0:
        return results

    tree_mg = KDTree(mg_coords)

    dists, _ = tree_mg.query(trac_coords, k=1)
    results['dist_to_mg'] = dists

    indices_list = tree_mg.query_ball_point(trac_coords, radius)
    counts = [len(idx) for idx in indices_list]
    results['mg_count'] = counts

    return results

# ==========================================
# ==========================================
print("...")
res_ctrl = analyze_trac_microglia_relation(adata_analyze, 'Control_Lateral', radius=200)
res_model = analyze_trac_microglia_relation(adata_analyze, 'Model_Lateral', radius=200)

# ==========================================
# ==========================================
fig, axs = plt.subplots(1, 2, figsize=(14, 6))

data_dist = []
if len(res_ctrl['dist_to_mg']) > 0:
    data_dist.extend([{'Distance': d, 'Group': 'Control'} for d in res_ctrl['dist_to_mg']])
if len(res_model['dist_to_mg']) > 0:
    data_dist.extend([{'Distance': d, 'Group': 'Model'} for d in res_model['dist_to_mg']])

df_dist = pd.DataFrame(data_dist)

if not df_dist.empty:
    sns.violinplot(data=df_dist, x='Group', y='Distance', ax=axs[0], palette=['blue', 'red'], inner='quartile')
    if len(res_ctrl['dist_to_mg']) > 0 and len(res_model['dist_to_mg']) > 0:
        s, p = ranksums(res_model['dist_to_mg'], res_ctrl['dist_to_mg'])
        sig = "***" if p<0.001 else ("*" if p<0.05 else "ns")
        title_sig = f"P={p:.2e} {sig}"
        mean_c = np.mean(res_ctrl['dist_to_mg'])
        mean_m = np.mean(res_model['dist_to_mg'])
        change = "⬇️ Closer" if mean_m < mean_c else "⬆️ Further"
    else:
        title_sig = "Insufficient Data"
        change = ""

    axs[0].set_title(f"Distance: {gene_target}+ Spot -> Nearest Microglia\n({change}, {title_sig})")
    axs[0].set_ylabel("Distance (Spatial Units)")
    axs[0].set_ylim(0, 1500)
else:
    axs[0].text(0.5, 0.5, "No Trac signals detected", ha='center')


data_count = []
if len(res_ctrl['mg_count']) > 0:
    data_count.extend([{'Count': c, 'Group': 'Control'} for c in res_ctrl['mg_count']])
if len(res_model['mg_count']) > 0:
    data_count.extend([{'Count': c, 'Group': 'Model'} for c in res_model['mg_count']])

df_count = pd.DataFrame(data_count)

if not df_count.empty:
    sns.boxplot(data=df_count, x='Group', y='Count', ax=axs[1], palette=['blue', 'red'], showfliers=False)
    if len(res_ctrl['mg_count']) > 0 and len(res_model['mg_count']) > 0:
        s, p = ranksums(res_model['mg_count'], res_ctrl['mg_count'])
        sig = "***" if p<0.001 else ("*" if p<0.05 else "ns")
        title_sig = f"P={p:.2e} {sig}"
        mean_c = np.mean(res_ctrl['mg_count'])
        mean_m = np.mean(res_model['mg_count'])
        change = "⬆️ More" if mean_m > mean_c else "⬇️ Less"
    else:
        title_sig = "Insufficient Data"
        change = ""

    axs[1].set_title(f"Microglia Count around {gene_target}+ Spots\n(Radius=200, {change}, {title_sig})")
    axs[1].set_ylabel("Number of Microglia Neighbors")
else:
    axs[1].text(0.5, 0.5, "No Trac signals detected", ha='center')

plt.tight_layout()
plt.show()

# ==========================================
# ==========================================
print("\n" + "="*60)
print(f"  📊 {gene_target}   ")
print("="*60)

if not df_dist.empty and not df_count.empty:
    print(f"{'Metric':<25} {'Control (Mean)':<15} {'Model (Mean)':<15} {'Diff':<10} {'P-value'}")
    print("-" * 75)

    mc_dist = df_dist[df_dist['Group']=='Control']['Distance'].mean()
    mm_dist = df_dist[df_dist['Group']=='Model']['Distance'].mean()
    _, p_dist = ranksums(res_model['dist_to_mg'], res_ctrl['dist_to_mg'])
    print(f"{'Distance to Nearest MG':<25} {mc_dist:.2f}           {mm_dist:.2f}           {mm_dist-mc_dist:+.2f}      {p_dist:.2e}")

    mc_count = df_count[df_count['Group']=='Control']['Count'].mean()
    mm_count = df_count[df_count['Group']=='Model']['Count'].mean()
    _, p_count = ranksums(res_model['mg_count'], res_ctrl['mg_count'])
    print(f"{'MG Neighbors (Count)':<25} {mc_count:.2f}           {mm_count:.2f}           {mm_count-mc_count:+.2f}      {p_count:.2e}")

    print("-" * 75)
    if mm_dist < mc_dist and p_dist < 0.05:
        print("✅ : Model side Trac+  (Spatial Attraction)。")
    if mm_count > mc_count and p_count < 0.05:
        print("✅ : Model side Trac+  (Clustering)。")
else:
    print("⚠️ ，。 Trac 。")


In [ ]:
# -----------------------------------------------------------------------------
# Cell 42: Analysis step
# -----------------------------------------------------------------------------
# Purpose:
# - Intermediate analysis step from the original notebook.
# - Consider refactoring into functions if reused.

# ==========================================
# ==========================================
target_gene = 'Trac'

if target_gene not in adata_lateral_new.var_names:
    if target_gene.upper() in adata_lateral_new.var_names:
        target_gene = target_gene.upper()
    elif 'Cd3e' in adata_lateral_new.var_names:
        print(f"⚠️  {target_gene}， Cd3e ...")
        target_gene = 'Cd3e'

print(f"🎯 Plotting: {target_gene}")

# ==========================================
# ==========================================
def get_spatial_layers(adata, group_name):
    subset = adata[adata.obs['split_group'] == group_name]

    coords_bg = subset.obsm['spatial']

    coords_mg = subset[subset.obs['cell_type'] == 'Microglia'].obsm['spatial']

    try:
        expr = subset[:, target_gene].X.toarray().flatten()
    except:
        expr = subset[:, target_gene].X.flatten()

    coords_target = subset.obsm['spatial'][expr > 0]

    return coords_bg, coords_mg, coords_target

# ==========================================
# ==========================================
fig, axs = plt.subplots(1, 2, figsize=(16, 8))
groups = ['Control_Lateral', 'Model_Lateral']
titles = ['Control Side (Right)', 'Model Side (Left)']

style_bg = {'c': '#e0e0e0', 's': 20, 'alpha': 0.3, 'label': 'Tissue Background'}
style_mg = {'c': '#9467bd', 's': 50, 'alpha': 0.4, 'label': 'Microglia'}       # 紫色
style_trac = {'c': '#d62728', 's': 150, 'marker': '*', 'edgecolors': 'black', 'linewidth': 0.5, 'label': f'{target_gene}+ Signal'} # 红色星星

for i, group in enumerate(groups):
    ax = axs[i]
    c_bg, c_mg, c_trac = get_spatial_layers(adata_lateral_new, group)

    ax.scatter(c_bg[:, 0], c_bg[:, 1], **style_bg)

    ax.scatter(c_mg[:, 0], c_mg[:, 1], **style_mg)

    if len(c_trac) > 0:
        ax.scatter(c_trac[:, 0], c_trac[:, 1], **style_trac)
        ax.scatter(c_trac[:, 0], c_trac[:, 1], s=300, facecolors='none', edgecolors='red', alpha=0.3)

    ax.set_title(f"{titles[i]}\n({target_gene}+ Spots: {len(c_trac)})", fontsize=14, fontweight='bold')
    ax.set_aspect('equal')
    ax.axis('off')

    if len(c_bg) > 0:
        pad = 200
        ax.set_xlim(c_bg[:, 0].min() - pad, c_bg[:, 0].max() + pad)
        ax.set_ylim(c_bg[:, 1].min() - pad, c_bg[:, 1].max() + pad)

    if i == 1:
        legend_elements = [
            Line2D([0], [0], marker='o', color='w', markerfacecolor=style_mg['c'], markersize=10, alpha=style_mg['alpha'], label='Microglia'),
            Line2D([0], [0], marker='*', color='w', markerfacecolor=style_trac['c'], markersize=15, markeredgecolor='k', label=f'{target_gene}+ T-Cell')
        ]
        ax.legend(handles=legend_elements, loc='upper right', fontsize=12)

plt.tight_layout()
plt.show()


In [ ]:
# -----------------------------------------------------------------------------
# Cell 43: Expression statistics (MWU / rank-based)
# -----------------------------------------------------------------------------
# Purpose:
# - Compare expression distributions between groups using non-parametric tests.
# - Report effect sizes and adjust p-values if many genes are tested.

# ==========================================
# ==========================================
adata_niche = adata_lateral_new.copy()
target_gene = 'Trac'
radius = 150

if target_gene not in adata_niche.var_names:
    if target_gene.upper() in adata_niche.var_names: target_gene = target_gene.upper()
    elif 'Cd3e' in adata_niche.var_names: target_gene = 'Cd3e'

print(f"🎯 : {target_gene}+ ")
print(f"⭕ : {radius}")

# ==========================================
# ==========================================
def analyze_niche_proportions(adata, group_name):
    subset = adata[adata.obs['split_group'] == group_name]

    try:
        expr = subset[:, target_gene].X.toarray().flatten()
    except:
        expr = subset[:, target_gene].X.flatten()

    t_coords = subset.obsm['spatial'][expr > 0]

    if len(t_coords) == 0:
        return pd.DataFrame()

    all_coords = subset.obsm['spatial']
    all_types = subset.obs['cell_type'].values
    tree = KDTree(all_coords)

    neighbors_list = tree.query_ball_point(t_coords, radius)

    niche_data = []

    for indices in neighbors_list:
        if len(indices) == 0: continue

        neighbor_types = all_types[indices]
        total_neighbors = len(neighbor_types)

        n_mg = np.sum(neighbor_types == 'Microglia')
        ratio_mg = (n_mg / total_neighbors) * 100

        n_astro = np.sum(neighbor_types == 'Astrocyte')
        n_da = np.sum(neighbor_types == 'DA_Neuron')
        n_oligo = np.sum(neighbor_types == 'Oligodendrocyte')

        niche_data.append({
            'Group': group_name,
            'Microglia_%': ratio_mg,
            'Astrocyte_%': (n_astro / total_neighbors) * 100,
            'DA_Neuron_%': (n_da / total_neighbors) * 100,
            'Oligodendrocyte_%': (n_oligo / total_neighbors) * 100,
            'Total_Neighbors': total_neighbors
        })

    return pd.DataFrame(niche_data)

# ==========================================
# ==========================================
df_ctrl = analyze_niche_proportions(adata_niche, 'Control_Lateral')
df_model = analyze_niche_proportions(adata_niche, 'Model_Lateral')

df_niche = pd.concat([df_ctrl, df_model], ignore_index=True)
df_niche['Group'] = df_niche['Group'].replace({'Control_Lateral': 'Control', 'Model_Lateral': 'Model'})

# ==========================================
# ==========================================
fig, axs = plt.subplots(1, 2, figsize=(14, 6))

if not df_niche.empty:
    sns.boxplot(data=df_niche, x='Group', y='Microglia_%', ax=axs[0],
                palette=['blue', 'red'], showfliers=False)
    sns.stripplot(data=df_niche, x='Group', y='Microglia_%', ax=axs[0],
                  color='black', alpha=0.3, size=3)

    if not df_ctrl.empty and not df_model.empty:
        s, p = ranksums(df_model['Microglia_%'], df_ctrl['Microglia_%'])
        sig = "***" if p<0.001 else ("*" if p<0.05 else "ns")
        title = f"Microglia % in Trac+ Niche\n(P={p:.2e} {sig})"
    else:
        title = "Insufficient Data"

    axs[0].set_title(title, fontsize=12, fontweight='bold')
    axs[0].set_ylabel("Percentage of Microglia (%)")
    axs[0].set_ylim(0, 100)

if not df_niche.empty:
    cols = ['Microglia_%', 'Astrocyte_%', 'DA_Neuron_%', 'Oligodendrocyte_%']
    mean_comp = df_niche.groupby('Group')[cols].mean()

    mean_comp['Other_%'] = 100 - mean_comp.sum(axis=1)

    mean_comp.plot(kind='bar', stacked=True, ax=axs[1],
                   color=['#9467bd', '#ff7f0e', '#d62728', '#2ca02c', '#d3d3d3'],
                   edgecolor='black')

    axs[1].set_title(f"Average Cellular Composition around {target_gene}+ Spots")
    axs[1].set_ylabel("Average Proportion (%)")
    axs[1].set_xticklabels(mean_comp.index, rotation=0)
    axs[1].legend(title='Neighbor Type', bbox_to_anchor=(1.05, 1))

plt.tight_layout()
plt.show()

# ==========================================
# ==========================================
print("\n" + "="*60)
print(f"  📊 {target_gene}+  (Niche Composition)")
print("="*60)

if not df_niche.empty:
    mean_c = df_ctrl['Microglia_%'].mean() if not df_ctrl.empty else 0
    mean_m = df_model['Microglia_%'].mean() if not df_model.empty else 0

    print(f"T :")
    print(f"   🔵 Control: {mean_c:.2f}%")
    print(f"   🔴 Model  : {mean_m:.2f}%")
    print(f"   📈    : {mean_m - mean_c:+.2f}%")

    if mean_m > mean_c:
        print("\n✅ : ，T 。")
        print(" T ，。")
    else:
        print("\n⚪ :  T 。")


In [ ]:
# -----------------------------------------------------------------------------
# Cell 44: Analysis step
# -----------------------------------------------------------------------------
# Purpose:
# - Intermediate analysis step from the original notebook.
# - Consider refactoring into functions if reused.

# ==========================================
# ==========================================
radius_um = 230
units_per_um = 1.0

radius_units = radius_um * units_per_um
crop_size = radius_units

adata_viz = adata_final.copy()
adata_viz.obs['Side'] = np.where(adata_viz.obsm['spatial'][:, 0] < 10000, 'Model', 'Control')

mg_df = pd.DataFrame(adata_viz[adata_viz.obs['cell_type'] == 'Microglia'].obsm['spatial'], columns=['x', 'y'])

# ==========================================
# ==========================================
def plot_niche_gallery(gene, side, max_plots=None):
    """
    
    """
    g_name = gene
    if g_name not in adata_viz.var_names and g_name.capitalize() in adata_viz.var_names:
        g_name = g_name.capitalize()

    try:
        expr = adata_viz[adata_viz.obs['Side'] == side, g_name].X.toarray().flatten()
    except:
        expr = adata_viz[adata_viz.obs['Side'] == side, g_name].X.flatten()

    t_coords = adata_viz[adata_viz.obs['Side'] == side].obsm['spatial'][expr > 0]

    n_total = len(t_coords)
    if n_total == 0:
        print(f"⚠️ {gene} ({side}): 。")
        return

    n_show = n_total if (max_plots is None) else min(n_total, max_plots)
    print(f"📸 {gene} ({side}):  {n_total} 。 {n_show} ...")

    n_cols = 5
    n_rows = int(np.ceil(n_show / n_cols))

    fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols*3, n_rows*3))
    axs = np.array(axs).flatten()

    star_color = '#d62728' if gene == 'Cd8b1' else '#1f77b4'

    for i in range(len(axs)):
        ax = axs[i]
        if i < n_show:
            center_x, center_y = t_coords[i]

            x_min, x_max = center_x - crop_size, center_x + crop_size
            y_min, y_max = center_y - crop_size, center_y + crop_size

            mask = (mg_df['x'] > x_min) & (mg_df['x'] < x_max) & \
                   (mg_df['y'] > y_min) & (mg_df['y'] < y_max)
            local_mg = mg_df[mask]

            ax.scatter(local_mg['x'], local_mg['y'],
                       c='#9467bd', s=22, alpha=0.70, linewidths=0)

            ax.scatter(center_x, center_y, c=star_color, s=200, marker='*',
                       edgecolors='white', linewidth=0.5)

            ax.set_xlim(x_min, x_max)
            ax.set_ylim(y_min, y_max)
            ax.set_aspect('equal')
            ax.axis('off')

            ax.set_title(f"Cell #{i+1}", fontsize=9)
        else:
            ax.axis('off')

    plt.suptitle(f"{gene} Microglia Niche (Radius 230um) - {side} Side", fontsize=16, y=1.01)
    plt.tight_layout()
    plt.show()


# ==========================================
# ==========================================

for gene in ['Cd4', 'Cd8b1']:
    for side in ['Model', 'Control']:
        plot_niche_gallery(gene, side, max_plots=20)


In [ ]:
# -----------------------------------------------------------------------------
# Cell 45: Analysis step
# -----------------------------------------------------------------------------
# Purpose:
# - Intermediate analysis step from the original notebook.
# - Consider refactoring into functions if reused.

rcParams['pdf.fonttype']        = 42
rcParams['ps.fonttype']         = 42
rcParams['font.family']         = 'serif'
rcParams['font.serif']          = ['Times New Roman', 'Times', 'DejaVu Serif']
rcParams['savefig.dpi']         = 300

out_dir = 'figures'
os.makedirs(out_dir, exist_ok=True)

def save_single_niche_dual(gene, side, cell_idx):
    """
     230 µm  PNG  PDF
    """
    g_name = gene.capitalize() if gene not in adata_viz.var_names else gene

    try:
        expr = adata_viz[adata_viz.obs['Side'] == side, g_name].X.toarray().flatten()
    except:
        expr = adata_viz[adata_viz.obs['Side'] == side, g_name].X.flatten()

    t_coords = adata_viz[adata_viz.obs['Side'] == side].obsm['spatial'][expr > 0]
    if cell_idx >= len(t_coords):
        raise IndexError(f'{gene} {side}  {len(t_coords)} ，idx={cell_idx} ！')

    center_x, center_y = t_coords[cell_idx]

    x_min, x_max = center_x - crop_size, center_x + crop_size
    y_min, y_max = center_y - crop_size, center_y + crop_size
    mask = (mg_df['x'] > x_min) & (mg_df['x'] < x_max) & \
           (mg_df['y'] > y_min) & (mg_df['y'] < y_max)
    local_mg = mg_df[mask]

    star_color = '#d62728' if gene == 'Cd8b1' else '#1f77b4'

    fig, ax = plt.subplots(figsize=(4, 4), facecolor='white')
    ax.set_facecolor('white')

    ax.scatter(local_mg['x'], local_mg['y'],
               c='#9467bd', s=40, alpha=0.7, lw=0)

    ax.scatter(center_x, center_y,
               c=star_color, s=150, marker='*',
               edgecolors='white', linewidth=0.8, zorder=10)

    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)
    ax.set_aspect('equal')
    ax.axis('off')

    fname_pdf = os.path.join(out_dir, f'{gene}_{side}_cell{cell_idx+1}.pdf')
    fig.savefig(fname_pdf, dpi=300, bbox_inches='tight', pad_inches=0.02,
                transparent=True)
    print(f'✅ PDF → {fname_pdf}')

    fname_png = os.path.join(out_dir, f'{gene}_{side}_cell{cell_idx+1}.png')
    fig.savefig(fname_png, dpi=600, bbox_inches='tight', pad_inches=0.02,
                facecolor='white', transparent=False)
    print(f'✅ PNG → {fname_png}')

    plt.close(fig)

targets = [
    ('Cd4',   'Model', 8),
    ('Cd4',   'Model', 13),
    ('Cd8b1', 'Model', 7),
    ('Cd8b1', 'Model', 8),
]

print("="*60)
print("figure（Times New Roman ）")
print("="*60)

for gene, side, idx in targets:
    save_single_niche_dual(gene, side, idx)

print("="*60)
print(f"✅ ！ {len(targets)*2}  '{out_dir}' ")
print("   - PDF: ，，")
print("   - PNG: 600 DPI ，， PPT/")
print("="*60)


In [ ]:
# -----------------------------------------------------------------------------
# Cell 46: Analysis step
# -----------------------------------------------------------------------------
# Purpose:
# - Intermediate analysis step from the original notebook.
# - Consider refactoring into functions if reused.

adata_analyze.obs["Side"] = adata_analyze.obs["condition"].astype(str)
print(adata_analyze.obs["Side"].value_counts())


In [ ]:
# -----------------------------------------------------------------------------
# Cell 47: Composition statistics (Fisher / Chi-square)
# -----------------------------------------------------------------------------
# Purpose:
# - Test whether cell-type composition differs between groups/regions.
# - Use Fisher's exact test for 2x2; use Chi-square for larger contingency tables.

# ==========================================
# ==========================================
rcParams['pdf.fonttype']   = 42
rcParams['ps.fonttype']    = 42
rcParams['font.family']    = 'serif'
rcParams['font.serif']     = ['Times New Roman', 'Times', 'DejaVu Serif']
rcParams['savefig.dpi']    = 300

out_dir = 'CD4_CD8_Analysis'
os.makedirs(out_dir, exist_ok=True)

# ==========================================
# ==========================================
genes_cd4 = ['Cd4']
genes_cd8 = ['Cd8b1']

def check_genes(g_list, var_names):
    valid = []
    for g in g_list:
        if g in var_names: valid.append(g)
        elif g.capitalize() in var_names: valid.append(g.capitalize())
    return list(set(valid))

valid_cd4 = check_genes(genes_cd4, adata_final.var_names)
valid_cd8 = check_genes(genes_cd8, adata_final.var_names)

print("="*80)
print("📊 CD4+ vs CD8+ T Cell Analysis Report")
print("="*80)
print(f"\n【】")
print(f"  CD4 : {valid_cd4}")
print(f"  CD8 : {valid_cd8}")

# ==========================================
# ==========================================
adata_t = adata_final.copy()

def get_expression_mask(adata, genes):
    if not genes: return np.zeros(adata.n_obs, dtype=bool)
    try:
        expr_sum = adata[:, genes].X.sum(axis=1).A1
    except:
        expr_sum = adata[:, genes].X.sum(axis=1)
    return expr_sum > 0

mask_cd4_pos = get_expression_mask(adata_t, valid_cd4)
mask_cd8_pos = get_expression_mask(adata_t, valid_cd8)

adata_t.obs['T_Subtype'] = 'Negative'
adata_t.obs.loc[mask_cd4_pos, 'T_Subtype'] = 'Cd4'
adata_t.obs.loc[mask_cd8_pos, 'T_Subtype'] = 'Cd8b1'
adata_t.obs.loc[mask_cd4_pos & mask_cd8_pos, 'T_Subtype'] = 'Cd4_Cd8b1_DP'

adata_t.obs['Side'] = np.where(adata_t.obsm['spatial'][:, 0] < 10000, 'Model', 'Control')

print(f"\n【】")
print(f"  n_units: {adata_t.n_obs:,}")
print(f"  CD4+ : {mask_cd4_pos.sum():,}")
print(f"  CD8+ : {mask_cd8_pos.sum():,}")
print(f"   (DP): {(mask_cd4_pos & mask_cd8_pos).sum():,}")
print(f"  : {(~mask_cd4_pos & ~mask_cd8_pos).sum():,}")

# ==========================================
# ==========================================
target_types = ['Cd4', 'Cd8b1', 'Cd4_Cd8b1_DP']
df_stats = []
stat_report = []

print("\n" + "="*80)
print("【】")
print("="*80)

for t in target_types:
    sub = adata_t[adata_t.obs['T_Subtype'] == t]
    n_ctrl = (sub.obs['Side'] == 'Control').sum()
    n_mod = (sub.obs['Side'] == 'Model').sum()

    total_ctrl = (adata_t.obs['Side'] == 'Control').sum()
    total_mod = (adata_t.obs['Side'] == 'Model').sum()

    pct_ctrl = 100 * n_ctrl / total_ctrl if total_ctrl > 0 else 0
    pct_mod = 100 * n_mod / total_mod if total_mod > 0 else 0

    # Fold Change
    fc = n_mod / (n_ctrl + 1e-9)

    contingency = [[n_ctrl, total_ctrl - n_ctrl],
                   [n_mod, total_mod - n_mod]]
    odds_ratio, p_value = fisher_exact(contingency)

    if p_value < 0.001:
        sig = "***"
    elif p_value < 0.01:
        sig = "**"
    elif p_value < 0.05:
        sig = "*"
    else:
        sig = "ns"

    df_stats.append({
        'Type': t,
        'Control': n_ctrl,
        'Model': n_mod,
        'FC': fc,
        'Pct_Ctrl': pct_ctrl,
        'Pct_Mod': pct_mod,
        'P_value': p_value,
        'Odds_Ratio': odds_ratio,
        'Significance': sig
    })

    print(f"\n【{t}】")
    print(f"  Control side:")
    print(f"    n_units: {n_ctrl:,} / {total_ctrl:,} ({pct_ctrl:.2f}%)")
    print(f"  Model side:")
    print(f"    n_units: {n_mod:,} / {total_mod:,} ({pct_mod:.2f}%)")
    print(f"  :")
    print(f"    Fold Change: {fc:.3f}×")
    print(f"    : {pct_mod - pct_ctrl:+.2f}%")
    print(f"    Odds Ratio: {odds_ratio:.3f}")
    print(f"    Fisher's exact test: P = {p_value:.4e} {sig}")

df_stats = pd.DataFrame(df_stats).set_index('Type')

stats_table_file = os.path.join(out_dir, 'CD4_CD8_Statistics.xlsx')
df_stats.to_excel(stats_table_file)
print(f"\n✅ : {stats_table_file}")

paper_table = []
for idx, row in df_stats.iterrows():
    paper_table.append({
        'Cell Type': idx,
        'Control n (%)': f"{int(row['Control'])} ({row['Pct_Ctrl']:.2f}%)",
        'Model n (%)': f"{int(row['Model'])} ({row['Pct_Mod']:.2f}%)",
        'Fold Change': f"{row['FC']:.2f}×",
        'Odds Ratio': f"{row['Odds_Ratio']:.2f}",
        'P-value': f"{row['P_value']:.2e}",
        'Significance': row['Significance']
    })

paper_df = pd.DataFrame(paper_table)
paper_file = os.path.join(out_dir, 'Table_For_Paper.xlsx')
paper_df.to_excel(paper_file, index=False)
print(f"✅ : {paper_file}")

# ==========================================
# ==========================================
print("\n" + "="*80)
print("【saved】")
print("="*80)

fig, ax = plt.subplots(figsize=(4.2, 3.2), facecolor='white')
ax.set_facecolor('white')

x = np.arange(len(df_stats))
w = 0.36

bars1 = ax.bar(x - w/2, df_stats['Control'], width=w, color='#4575B4',
               linewidth=0, label='Control')
bars2 = ax.bar(x + w/2, df_stats['Model'], width=w, color='#D73027',
               linewidth=0, label='Model')

ax.set_xticks(x)
ax.set_xticklabels(df_stats.index, fontsize=10)
ax.set_ylabel('Number of Cells', fontsize=10)
ax.set_title('CD4 vs CD8 Cell Counts', fontsize=11, fontweight='bold')
ax.legend(frameon=False, fontsize=9)
ax.grid(False)

for s in ['top','right','left','bottom']:
    ax.spines[s].set_visible(False)

for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}',
                ha='center', va='bottom', fontsize=8)

plt.tight_layout()

pdf_file = os.path.join(out_dir, 'Cd4_Cd8b1_Counts.pdf')
png_file = os.path.join(out_dir, 'Cd4_Cd8b1_Counts.png')
plt.savefig(pdf_file, bbox_inches='tight', transparent=True)
plt.savefig(png_file, dpi=600, bbox_inches='tight', facecolor='white')
plt.show()
print(f"✅ :")
print(f"   - {pdf_file}")
print(f"   - {png_file}")

# ==========================================
# ==========================================
def plot_spatial_single(adata_all, subtype, color, size=28, mid_x=10000,
                        out_prefix='Spatial', rotate180=True):
    fig, ax = plt.subplots(figsize=(5.2, 4.2), facecolor='white')
    ax.set_facecolor('white')

    xy = adata_all.obsm['spatial'].copy()

    if rotate180:
        xmax, ymax = xy[:, 0].max(), xy[:, 1].max()
        xy[:, 0] = xmax - xy[:, 0]
        xy[:, 1] = ymax - xy[:, 1]
        mid_plot = xmax - mid_x
    else:
        mid_plot = mid_x

    ax.scatter(xy[:, 0], xy[:, 1], s=8, c='#D9D9D9', alpha=0.18, linewidths=0)

    sub = adata_all[adata_all.obs['T_Subtype'] == subtype]
    n_total = sub.n_obs

    if n_total > 0:
        xy2 = sub.obsm['spatial'].copy()
        if rotate180:
            xy2[:, 0] = xmax - xy2[:, 0]
            xy2[:, 1] = ymax - xy2[:, 1]
        ax.scatter(xy2[:, 0], xy2[:, 1], s=size, c=color, alpha=0.95,
                   edgecolors='white', linewidths=0.2)

    ax.axvline(mid_plot, color='black', linestyle='--', lw=1, alpha=0.5)
    ax.set_title(f'Spatial Distribution: {subtype} (n={n_total})',
                 fontsize=11, fontweight='bold')
    ax.set_axis_off()
    plt.tight_layout()

    pdf_name = os.path.join(out_dir, f"{out_prefix}_{subtype}_rot180.pdf")
    png_name = os.path.join(out_dir, f"{out_prefix}_{subtype}_rot180.png")
    plt.savefig(pdf_name, bbox_inches='tight', transparent=True)
    plt.savefig(png_name, dpi=600, bbox_inches='tight', facecolor='white')
    plt.show()

    print(f"✅ :")
    print(f"   - {pdf_name}")
    print(f"   - {png_name}")


plot_spatial_single(adata_t, 'Cd4',   '#1f77b4', size=28)
plot_spatial_single(adata_t, 'Cd8b1', '#d62728', size=36)
